In [64]:
import os
import re
from problog.program import PrologString
from problog import get_evaluatable
import csv
import pandas as pd 
import signal
import time
import multiprocessing
import os
import glob
import re

In [65]:
directory_name = "../outputMIX2S/prob-rules" 
api_file_name = directory_name + "/apis_with_ids.csv"

In [66]:


def find_files_with_prefix(directory, prefix):
    content_array = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith(prefix):
                with open(os.path.join(root, file), 'r') as f:
                    content_array.extend(f.readlines())
    return content_array
# Example usage

content_array = find_files_with_prefix(directory_name, "output")
cleaned_content = [line.replace("\n", "").replace("\t", "").strip() for line in content_array]
cleaned_content

#'fapi_ac(rapi1244,no_ac):9.9e-05',

 

['fapi_ac(rapi1202,X2):0',
 'fapi_ac(rapi1203,normal):0',
 'fapi_ac(rapi601,X2):0',
 'fapi_ac(rapi604,X2):0',
 'fapi_ac(rapi602,X2):0',
 'fapi_ac(rapi599,X2):0',
 'fapi_ac(rapi603,X2):0',
 'fapi_ac(rapi600,X2):0',
 'fapi_ac(rapi380,X2):0',
 'fapi_ac(rapi381,X2):0',
 'fapi_ac(rapi2523,X2):0',
 'fapi_ac(rapi2526,X2):0',
 'fapi_ac(rapi2524,X2):0',
 'fapi_ac(rapi2525,X2):0',
 'fapi_ac(rapi152,no_ac):0',
 'fapi_ac(rapi1213,X2):0',
 'fapi_ac(rapi1214,X2):0',
 'fapi_ac(rapi1211,X2):0',
 'fapi_ac(rapi1206,X2):0',
 'fapi_ac(rapi1205,X2):0',
 'fapi_ac(rapi1204,X2):0',
 'fapi_ac(rapi1210,X2):0',
 'fapi_ac(rapi1207,X2):0',
 'fapi_ac(rapi1208,X2):0',
 'fapi_ac(rapi1212,X2):0',
 'fapi_ac(rapi1209,X2):0',
 'fapi_ac(rapi592,X2):0',
 'fapi_ac(rapi2478,X2):0',
 'fapi_ac(rapi2477,X2):0',
 'fapi_ac(rapi2473,X2):0',
 'fapi_ac(rapi2475,X2):0',
 'fapi_ac(rapi2476,X2):0',
 'fapi_ac(rapi2474,X2):0',
 'fapi_ac(rapi2472,X2):0',
 'fapi_ac(rapi577,X2):0',
 'fapi_ac(rapi591,X2):0',
 'fapi_ac(rapi583,X2):0',
 'fapi_

In [67]:
extracted_data = []

for line in cleaned_content:
    match = re.search(r"fapi_ac\(rapi(\d+),(\w+)\):([0-9.e-]+)", line)
    if match:
        rapi_number, system_x2, value = match.groups()
        if value != "0":
            extracted_data.append(("api" + rapi_number, system_x2, value))

print(extracted_data)

[('api1246', 'system', '0.096144171'), ('api1245', 'system', '0.096144171'), ('api1253', 'system', '0.26303483'), ('api1244', 'system', '0.018150404'), ('api1260', 'system', '0.26303483'), ('api1148', 'normal', '0.141312'), ('api1083', 'normal', '0.141312'), ('api1149', 'normal', '0.141312'), ('api16', 'no_ac', '0.036001225'), ('api15', 'no_ac', '0.036001225'), ('api0', 'no_ac', '0.046885128'), ('api21', 'system', '0.2469894'), ('api22', 'system', '0.2469894'), ('api128', 'normal', '0.1824'), ('api124', 'normal', '0.1824'), ('api120', 'normal', '0.161196'), ('api123', 'normal', '0.161196'), ('api2583', 'system', '0.36035621'), ('api2585', 'system', '0.35521155'), ('api2574', 'system', '0.26455392'), ('api2586', 'system', '0.42373447'), ('api2587', 'system', '0.42373447'), ('api2588', 'system', '0.059710134'), ('api1575', 'system', '0.44686361'), ('api1552', 'system', '0.46460334'), ('api1572', 'system', '0.44686361'), ('api1573', 'system', '0.44686361'), ('api401', 'system', '0.0488910

In [68]:
highest_values = {}
for number, category, value in extracted_data:
    value = float(value)  # Convert the value to float for comparison
    if number not in highest_values or value > highest_values[number][1]:
        highest_values[number] = (category, value)

highest_values

{'api1246': ('system', 0.096144171),
 'api1245': ('system', 0.096144171),
 'api1253': ('system', 0.26303483),
 'api1244': ('system', 0.018150404),
 'api1260': ('system', 0.26303483),
 'api1148': ('normal', 0.141312),
 'api1083': ('normal', 0.141312),
 'api1149': ('normal', 0.141312),
 'api16': ('no_ac', 0.036001225),
 'api15': ('no_ac', 0.036001225),
 'api0': ('no_ac', 0.046885128),
 'api21': ('system', 0.2469894),
 'api22': ('system', 0.2469894),
 'api128': ('normal', 0.1824),
 'api124': ('normal', 0.1824),
 'api120': ('normal', 0.161196),
 'api123': ('normal', 0.161196),
 'api2583': ('system', 0.36035621),
 'api2585': ('system', 0.35521155),
 'api2574': ('system', 0.26455392),
 'api2586': ('system', 0.42373447),
 'api2587': ('system', 0.42373447),
 'api2588': ('system', 0.059710134),
 'api1575': ('system', 0.44686361),
 'api1552': ('system', 0.46460334),
 'api1572': ('system', 0.44686361),
 'api1573': ('system', 0.44686361),
 'api401': ('system', 0.048891059),
 'api400': ('system', 0

In [69]:
inferenceDf = pd.DataFrame([(number, cat, val) for number, (cat, val) in highest_values.items()],
                  columns=['api', 'protection', 'Probability'])

# Write to CSV
csv_filename = './inference.csv'
df.to_csv(csv_filename, index=False)

print(f"Data written to {csv_filename}")
inferenceDf



Data written to ./inference.csv


,api,protection,Probability
0,api1246,system,0.096144
1,api1245,system,0.096144
2,api1253,system,0.263035
3,api1244,system,0.018150
4,api1260,system,0.263035
...,...,...,...
257,api1386,system,0.981000
258,api1424,system,0.742550
259,api1412,system,0.087994
260,api1432,system,0.446652


In [70]:
column_names = [i for i in range(0, 100)]
originalApiDf = pd.read_csv(api_file_name, header=None, names=column_names)


rename_dict = {
    0: 'api',
    2: 'protection',
}

originalApiDf.rename(columns=rename_dict, inplace=True)

originalApiDf['protection'] = originalApiDf['protection'].replace({
    "NONE": "no_ac",
    "SYS_OR_SIG" : "system",
    "NORMAL" : "normal",
})

originalApiDf


,api,1,protection,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,api22,com.miui.server.PerfShielderService.setService...,system,field32::packageName::Ljava/lang/String::modif...,field29::checkPriority::Z::modify::some,field34::serviceName::Ljava/lang/String::modif...,field33::priority::I::modify::all,field53::ALL_MEMBERS::Ljava/lang/String::modif...,field30::delayTime::J::modify::all,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,api21,com.miui.server.PerfShielderService.setService...,system,field32::packageName::Ljava/lang/String::modif...,field29::checkPriority::Z::modify::some,field34::serviceName::Ljava/lang/String::modif...,field33::priority::I::modify::all,field53::ALL_MEMBERS::Ljava/lang/String::modif...,field30::delayTime::J::modify::all,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,api16,com.miui.server.PerfShielderService.reportPerc...,no_ac,field259::ALL_MEMBERS::Ljava/lang/String::modi...,field258::ALL_MEMBERS::Ljava/lang/String::fetc...,field259::ALL_MEMBERS::Ljava/lang/String::fetc...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,api15,com.miui.server.PerfShielderService.reportAnr(...,no_ac,field259::ALL_MEMBERS::Ljava/lang/String::modi...,field258::ALL_MEMBERS::Ljava/lang/String::fetc...,field259::ALL_MEMBERS::Ljava/lang/String::fetc...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,api13,com.miui.server.PerfShielderService.removeServ...,system,field32::packageName::Ljava/lang/String::fetch...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,api2587,com.android.server.LocationPolicyManagerServic...,system,field7318::ALL_MEMBERS::I::fetch::all,field7313::ALL_MEMBERS::I::fetch::all,field7323::ALL_MEMBERS::I::fetch::all,field7313::ALL_MEMBERS::I::modify::all,field7323::ALL_MEMBERS::I::modify::all,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
775,api2588,com.android.server.LocationPolicyManagerServic...,system,field7318::ALL_MEMBERS::I::fetch::all,field7313::ALL_MEMBERS::I::fetch::all,field7323::ALL_MEMBERS::I::modify::all,field7269::mHighCost::Z::fetch::some,field7271::mMinIntervalMs::I::fetch::some,field7272::mProvider::Ljava/lang/String::fetch...,field7318::ALL_MEMBERS::I::add::some,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776,api2594,com.android.server.pm.OtaDexoptService.nextDex...,no_ac,field7329::ALL_MEMBERS::Ljava/lang/String::mod...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,api2590,com.android.server.pm.OtaDexoptService.cleanup()V,no_ac,field827::ALL_MEMBERS::Ljava/lang/Object::modi...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
merged_df = pd.merge(originalApiDf, inferenceDf, on=['api'], suffixes=('_actual', '_infer'))
mismatches = merged_df[merged_df['protection_actual'] != merged_df['protection_infer']]

# mismatches

# mismatches.drop(columns=['protection_actual', 'Probability'], inplace=True)

mismatches.to_csv('./mismatchesMIX2woOPRULESZTE.csv', index=False)

In [72]:
# mismatches['api', 1,'protection']
mismatches

,api,1,protection_actual,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,protection_infer,Probability
16,api457,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::modify::all,field2746::ALL_MEMBERS::I::fetch::some,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.654783
17,api468,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
21,api452,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
22,api459,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
24,api465,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
25,api467,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
26,api451,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
28,api466,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
29,api456,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
30,api464,com.android.server.print.PrintManagerService$P...,no_ac,field2746::ALL_MEMBERS::I::fetch::all,field2746::ALL_MEMBERS::I::modify::all,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,system,0.778307
